In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL
import os
import re
import cv2

In [2]:
df = pd.read_csv('tracking.txt', sep = '\t')

In [3]:
X_centers = df[df.id == 0]['xBody'].values
y_centers = df[df.id == 0]['yBody'].values
img_numbers = df[df.id == 0]['imageNumber'].values

In [4]:
data = []
c1 = 0
for dirname, _, filenames in (os.walk('video/images/')):
    for c, filename in enumerate(sorted(filenames)):
        fr = int(re.findall(r'\d+', filename)[0])
        if (fr in img_numbers):
            img = PIL.Image.open(os.path.join(dirname, filename)).convert('L')
            g = np.asarray(img)
            ind = np.where(img_numbers == fr)[0].item()
            X = int(X_centers[ind])
            Y = int(y_centers[ind])
            im = g[Y-40 : Y+40, X-40 : X+40].astype(np.uint8)
            data.append(im)
            
data = np.array(data)

In [5]:

for d, n in zip(data, img_numbers):
    PIL.Image.fromarray(d).save('test_data/frame' + str(n) + '.jpg')